In [127]:
import numpy as np
import pandas as pd
import matplotlib as plt
from ucimlrepo import fetch_ucirepo 

In [128]:
! pip install ucimlrepo


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


Implementação Perceptron Clássico

In [124]:
def loader_breast_cancer():
        breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

        X = breast_cancer_wisconsin_diagnostic.data.features.to_numpy()
        y = breast_cancer_wisconsin_diagnostic.data.targets.iloc[:, 0].to_numpy()

        y = np.where(y == "M", 1, 0)



        return X, y


Pre-Processamento dos Datasets

In [109]:
def stratified_kfold(X, y, n_splits=5):
    classes = np.unique(y)
    idx_splits = [[] for _ in range(n_splits)]

    for cls in classes:
        cls_idx = np.where(y == cls)[0]
        np.random.shuffle(cls_idx)
        cls_splits = np.array_split(cls_idx, n_splits)
        for i in range(n_splits):
            idx_splits[i].extend(cls_splits[i])

    folds = []
    for i in range(n_splits):
        test_idx = np.array(idx_splits[i])
        train_idx = np.array([idx for j, fold in enumerate(idx_splits) if j != i for idx in fold])
        folds.append((train_idx, test_idx))

    return folds

def z_score(X_train, X_test):
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    std[std == 0] = 1
    X_train_scaled = (X_train - mean) / std
    X_test_scaled = (X_test - mean) / std
    return X_train_scaled, X_test_scaled        



In [132]:
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix_manual(y_true, y_pred, classes=[0, 1], normalize=False, title="Matriz de Confusão"):
    """
    Plota a matriz de confusão usando matplotlib, sem sklearn.
    
    Parâmetros:
    - y_true: rótulos verdadeiros
    - y_pred: rótulos preditos
    - classes: lista de classes
    - normalize: se True, mostra proporção
    - title: título do gráfico
    """
    n_classes = len(classes)
    cm = np.zeros((n_classes, n_classes), dtype=int)
    class_to_index = {cls: i for i, cls in enumerate(classes)}

    # Preencher a matriz
    for true, pred in zip(y_true, y_pred):
        i = class_to_index[true]
        j = class_to_index[pred]
        cm[i, j] += 1

    if normalize:
        cm = cm.astype(float) / cm.sum(axis=1)[:, np.newaxis]

    # Plotar com matplotlib
    fig, ax = plt.subplots(figsize=(5, 5))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)

    # Mostrar os números dentro das células
    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    ax.set_xlabel("Predito")
    ax.set_ylabel("Verdadeiro")
    ax.set_xticks(np.arange(n_classes))
    ax.set_yticks(np.arange(n_classes))
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)
    ax.set_title(title)
    plt.show()


Implementação Perceptron Clássico

In [114]:
class Perceptron:
    def __init__(self, epochs=100, eta=0.1):
        self.epochs = epochs
        self.eta = eta
        self.w = None

    def fit(self, X, y):
        n, d = X.shape
        self.w = np.zeros(d + 1)  # inclui bias

        # Converter labels para -1 e 1
        y_binary = np.where(y == 0, -1, 1)

        for epoch in range(self.epochs):
            errors = 0
            indices = np.arange(n)
            np.random.shuffle(indices)

            for i in indices:
                x_i = np.append(X[i], 1) 
                if y_binary[i] * np.dot(self.w, x_i) <= 0:
                    self.w += self.eta * y_binary[i] * x_i
                    errors += 1

            if errors == 0:
                print(f"Convergência na época {epoch+1}")
                break

    def decision_function(self, X):
        X_bias = np.c_[X, np.ones(X.shape[0])]
        return X_bias @ self.w

    def predict(self, X):
        pred = np.sign(self.decision_function(X))
        return np.where(pred == -1, 0, 1)  # volta para 0/1

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)



In [69]:
def compute_macro_f1(y_true, y_pred, n_classes=None):
    if n_classes is None:
        n_classes = len(np.unique(np.concatenate([y_true, y_pred])))
    
    f1_scores = []
    
    for i in range(n_classes):
        tp = np.sum((y_true == i) & (y_pred == i))
        fp = np.sum((y_true != i) & (y_pred == i))
        fn = np.sum((y_true == i) & (y_pred != i))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

In [131]:
def run_stratified_cross_validation():
    
    X, y = loader_breast_cancer()
    
    param_grid = {
        "epochs": [500, 1000],
        "learning_rate": [0.001, 0.01, 0.1]
    }
    
    k_folds = 5
    best_result = None
    results = []
    
    folds = stratified_kfold(X, y, n_splits=k_folds)
    
    for epochs in param_grid["epochs"]:
        for learning_rate in param_grid["learning_rate"]:
            
            accuracies_test = []
            f1_scores = []
            
            for fold, (train_idx, test_idx) in enumerate(folds):
                X_train, X_test = X[train_idx], X[test_idx]
                y_train, y_test = y[train_idx], y[test_idx]

                X_train_scaled, X_test_scaled = z_score(X_train, X_test)

                model = Perceptron(epochs=epochs, eta=learning_rate)
                model.fit(X_train_scaled, y_train)
                
                test_accuracy = model.accuracy(X_test_scaled, y_test)
                accuracies_test.append(test_accuracy)

                y_pred = model.predict(X_test_scaled)
                f1 = compute_macro_f1(y_test, y_pred)
                f1_scores.append(f1)
            
            mean_test = np.mean(accuracies_test)
            std_test = np.std(accuracies_test)
            mean_f1 = np.mean(f1_scores)
            std_f1 = np.std(f1_scores)
            
            print(f"epochs={epochs}, lr={learning_rate:.3f} -> "
                  f"Acurácia TESTE: {mean_test:.4f} ± {std_test:.4f} | "
                  f"Macro-F1: {mean_f1:.4f} ± {std_f1:.4f}")
            
            results.append(((epochs, learning_rate), mean_test, std_test, mean_f1, std_f1))
            
            if best_result is None or mean_test > best_result[1]:
                best_result = ((epochs, learning_rate), mean_test, std_test, mean_f1, std_f1)
    
    best_params, best_mean, best_std, best_f1, best_f1_std = best_result
    print(f"\nMelhor configuração: epochs={best_params[0]}, lr={best_params[1]} "
          f"-> Acurácia {best_mean:.4f} ± {best_std:.4f} | "
          f"Macro-F1 {best_f1:.4f} ± {best_f1_std:.4f}")

    # --- Matriz de confusão usando o último fold do melhor parâmetro ---
    # Treinar novamente no último fold do melhor conjunto de parâmetros
    last_train_idx, last_test_idx = folds[-1]
    X_train, X_test = X[last_train_idx], X[last_test_idx]
    y_train, y_test = y[last_train_idx], y[last_test_idx]
    X_train_scaled, X_test_scaled = z_score(X_train, X_test)

    best_model = Perceptron(epochs=best_params[0], eta=best_params[1])
    best_model.fit(X_train_scaled, y_train)
    y_pred = best_model.predict(X_test_scaled)

    # Chamar a matriz de confusão manual
    plot_confusion_matrix_manual(y_test, y_pred, classes=[0, 1], normalize=False)

    return best_result

# --- Execução ---
if __name__ == "__main__":
    best_params = run_stratified_cross_validation()

Convergência na época 146
Convergência na época 446
epochs=500, lr=0.001 -> Acurácia TESTE: 0.9579 ± 0.0128 | Macro-F1: 0.9552 ± 0.0133
Convergência na época 100
Convergência na época 350
epochs=500, lr=0.010 -> Acurácia TESTE: 0.9596 ± 0.0044 | Macro-F1: 0.9569 ± 0.0044
Convergência na época 108
Convergência na época 381
epochs=500, lr=0.100 -> Acurácia TESTE: 0.9596 ± 0.0069 | Macro-F1: 0.9571 ± 0.0072
Convergência na época 78
Convergência na época 280
epochs=1000, lr=0.001 -> Acurácia TESTE: 0.9562 ± 0.0144 | Macro-F1: 0.9534 ± 0.0150
Convergência na época 98
Convergência na época 248
epochs=1000, lr=0.010 -> Acurácia TESTE: 0.9596 ± 0.0117 | Macro-F1: 0.9570 ± 0.0123
Convergência na época 117
Convergência na época 320
epochs=1000, lr=0.100 -> Acurácia TESTE: 0.9596 ± 0.0069 | Macro-F1: 0.9570 ± 0.0071

Melhor configuração: epochs=1000, lr=0.01 -> Acurácia 0.9596 ± 0.0117 | Macro-F1 0.9570 ± 0.0123
Convergência na época 408
Matriz de Confusão:
Pred→	0	1
Verdadeiro 0:	67	4
Verdadeiro